In [1]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [3]:
from langchain_openai import ChatOpenAI


In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


In [5]:


class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [5]:
message = {
    "role": "user",
    "content": "Who walked on the moon for the first time? Print only the name."
}

response = graph.invoke({"messages": [message]})
print(response["messages"][-1].content)


Neil Armstrong


In [ ]:
state = None

while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break

    if state is None:
        state = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append(
            {"role": "user", "content": in_message}
        )

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)


Bot: I don't have real-time data access to provide current stock prices. To find the latest price of Apple Inc. (AAPL), please check a financial news website, a stock market app, or a brokerage platform.
Bot: The first person to walk on the Moon was astronaut Neil Armstrong. He took his historic step on July 20, 1969, during NASA's Apollo 11 mission. Armstrong famously declared, "That's one small step for [a] man, one giant leap for mankind," as he set foot on the lunar surface.
Bot: Neil Armstrong walked on the Moon on July 20, 1969. This event marked a significant milestone in space exploration and was part of NASA's Apollo 11 mission.
Bot: It seems like your message might have been cut off or sent accidentally. If you have any more questions or need further information, feel free to ask!


In [7]:
import gradio as gr

def chat(user_input, history):
    result = graph.invoke({
        "messages": [
            {"role": "user", "content": user_input}
        ]
    })
    return result["messages"][-1].content


gr.ChatInterface(
    fn=chat,
    title="Simple LangGraph Chatbot",
    description="LangGraph chatbot running inside a notebook"
).launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
